<a href="https://colab.research.google.com/github/lynaBoukari/NSFW_image_model_checker/blob/main/NSFW_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install scikit-learn


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,640 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,846 kB]
Get:13 https://developer.download.nvidia.com/compute/cud

In [2]:
!pip install easyocr opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 12.5 MB/s eta 0:00:00


# Safety-checker using Bernoulli Naive Bayes model and OCR



In [3]:
import cv2
import numpy as np
from PIL import Image
import pytesseract
import easyocr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import  BernoulliNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import os
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Preprocessing images before text extraction

def preprocess_image(image_path):
    # #Load the image
    img = cv2.imread(image_path)

     # # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # # Threshold to create a binary image
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # # Invert the binary image
    inverted = cv2.bitwise_not(thresh)

    # # Save preprocessed image for debugging
    cv2.imwrite('ProcessedImage.png', inverted)

    return Image.fromarray(inverted)


# Function to extract text using Tesseract
def extract_text_tesseract(image_path):
    preprocessed_img = preprocess_image(image_path)
    custom_config = r'--oem 3 --psm 6'
    return pytesseract.image_to_string(preprocessed_img, config=custom_config)

# # Function to extract text using EasyOCR
def extract_text_easyocr(image_path):
    reader = easyocr.Reader(['en'], gpu=False) #choosing english
    preprocessed_img = preprocess_image(image_path)
    img_array = np.array(preprocessed_img)
    results = reader.readtext(img_array)
    return ' '.join([res[1] for res in results])

# Extracting text using the best OCR
def extract_text(image_path):
    text_tesseract = extract_text_tesseract(image_path)
    text_easyocr = extract_text_easyocr(image_path)

    # Choose the best result based on longer length
    return text_easyocr if len(text_easyocr) > len(text_tesseract) else text_tesseract

# Text preprocessing
def preprocess_text(text):
    text = text.lower() # Lower case charachters
    text = re.sub(r'[^a-z\s]', '', text)  # Remove non-alphabetic characters
    text = ' '.join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])
    text = ' '.join(text.split()) # Remove spaces
    return text


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Classification Report:
              precision    recall  f1-score   support

           0       0.17      0.20      0.18         5
           1       0.67      0.62      0.64        13

    accuracy                           0.50        18
   macro avg       0.42      0.41      0.41        18
weighted avg       0.53      0.50      0.51        18

Accuracy: 0.50


Data loading, Training and evaluation

In [11]:

# Prepare dataset
def prepare_dataset(data_dir):
    texts, labels, fonts = [], [], []

    for filename in os.listdir(data_dir):
        if filename.endswith('.png'):
            file_path = os.path.join(data_dir, filename)

            img = Image.open(file_path)
            metadata = img.info

            extracted_text = extract_text(file_path)
            cleaned_text = preprocess_text(extracted_text)

            texts.append(cleaned_text)
            labels.append(1 if metadata['nsfw'] == 'True' else 0)
            fonts.append(metadata['font'])

    return texts, labels, fonts

# Train and evaluate the model
def train_and_evaluate(data_dir):
    texts, labels, _ = prepare_dataset(data_dir)
    X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

    # Using a Bernouli Naives Bayes classifier with a tfidf feature extractor
    model = make_pipeline(TfidfVectorizer(), BernoulliNB())
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

    return model

# Training and evaluation on dataset_image_train
data_dir = '/content/Data'
model = train_and_evaluate(data_dir)


Classification Report:
              precision    recall  f1-score   support

           0       0.17      0.20      0.18         5
           1       0.67      0.62      0.64        13

    accuracy                           0.50        18
   macro avg       0.42      0.41      0.41        18
weighted avg       0.53      0.50      0.51        18

Accuracy: 0.50


In [9]:
import joblib

# Saving the model to a file for later use
joblib.dump(model, 'NSWF_checker.pkl')
print("Model saved as NSWF_checker.pkl")


Model saved as NSWF_checker.pkl


## Predict on a new image

In [4]:
def predict_nsfw(image_path,model):
    # Extract text from the image
    extracted_text = extract_text(image_path)
    cleaned_text = preprocess_text(extracted_text)

    # Predict using the trained model
    prediction = model.predict([cleaned_text])

    if prediction == [True]:
        print("The text in the image is NSFW.")
    else:
        print("The text in the image is safe.")


In [19]:
# Loading the model for test
model = joblib.load('/content/NSWF_checker.pkl')


# Prediction pipline
def predict_nsfw(image_path,model):
    # Extract text from the image
    extracted_text = extract_text(image_path)
    cleaned_text = preprocess_text(extracted_text)

    # Predict using the trained model
    prediction = model.predict([cleaned_text])

    if prediction == [True]:
        print("The text in the image is NSFW.")
    else:
        print("The text in the image is safe.")


# Use the loaded model for prediction
predict_nsfw('/content/Data/img_14.png', model)


The text in the image is NSFW.


# Validation function

Preprcessing functions

In [12]:
import cv2
import numpy as np
from PIL import Image
import pytesseract
import easyocr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import  BernoulliNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import os
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Preprocessing images before text extraction

def preprocess_image(image_path):
    # #Load the image
    img = cv2.imread(image_path)

     # # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # # Threshold to create a binary image
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # # Invert the binary image
    inverted = cv2.bitwise_not(thresh)

    # # Save preprocessed image for debugging
    cv2.imwrite('ProcessedImage.png', inverted)

    return Image.fromarray(inverted)


# Function to extract text using Tesseract
def extract_text_tesseract(image_path):
    preprocessed_img = preprocess_image(image_path)
    custom_config = r'--oem 3 --psm 6'
    return pytesseract.image_to_string(preprocessed_img, config=custom_config)

# # Function to extract text using EasyOCR
def extract_text_easyocr(image_path):
    reader = easyocr.Reader(['en'], gpu=False) #choosing english
    preprocessed_img = preprocess_image(image_path)
    img_array = np.array(preprocessed_img)
    results = reader.readtext(img_array)
    return ' '.join([res[1] for res in results])

# Extracting text using the best OCR
def extract_text(image_path):
    text_tesseract = extract_text_tesseract(image_path)
    text_easyocr = extract_text_easyocr(image_path)

    # Choose the best result based on longer length
    return text_easyocr if len(text_easyocr) > len(text_tesseract) else text_tesseract

# Text preprocessing
def preprocess_text(text):
    text = text.lower() # Lower case charachters
    text = re.sub(r'[^a-z\s]', '', text)  # Remove non-alphabetic characters
    text = ' '.join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])
    text = ' '.join(text.split()) # Remove spaces
    return text


Preprocessing of validation images and Evaluation of the trained model function

In [14]:
# Prepare validation dataset and evaluate the model
def prepare_and_evaluate(validation_dir, model):

    texts, labels = [], []

    # Prepare the dataset
    for filename in os.listdir(validation_dir):
        if filename.endswith('.png'):
            file_path = os.path.join(validation_dir, filename)

            # Open image and read metadata
            img = Image.open(file_path)
            metadata = img.info

            # Extract and preprocess text
            extracted_text = extract_text(file_path)
            cleaned_text = preprocess_text(extracted_text)

            # Append to dataset
            texts.append(cleaned_text)
            labels.append(1 if metadata.get('nsfw') == 'True' else 0)

    # Evaluate the model
    predictions = model.predict(texts)

    print("Classification Report:")
    print(classification_report(labels, predictions))
    accuracy = accuracy_score(labels, predictions)
    print(f"Accuracy: {accuracy:.2f}")

    return accuracy


Main

In [15]:
# Load the trained model
import joblib
model = joblib.load('/content/NSWF_checker.pkl')

# Path to the validation directory
validation_dir = '/content/Data'

# Prepare and evaluate
prepare_and_evaluate(validation_dir, model)


Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.83      0.85        42
           1       0.86      0.89      0.88        47

    accuracy                           0.87        89
   macro avg       0.87      0.86      0.86        89
weighted avg       0.87      0.87      0.86        89

Accuracy: 0.87


0.8651685393258427